## Notebook #1: Entendendo os dados (CX Analytics)

##### Este Notebook inicial tem como objetivo compreender o dataset (Brasilian E-Commerce Dataset (Olist)) para posteriormente começar as análises de CX. Os principais pontos que farão diferença no resultado final se refere à dados referentes aos consumidores, as avaliações e informações dos pedidos.

Primeiro, faz-se importações necessárias e se passa as tableas para as variáveis correspondentes:

In [4]:
# Importação das bibliotecas necessárias
import pandas as pd
import numpy as np

In [ ]:
orders = pd.read_csv("../data/raw/olist_orders_dataset.csv")
reviews = pd.read_csv("../data/raw/olist_order_reviews_dataset.csv")
customers = pd.read_csv("../data/raw/olist_customers_dataset.csv")

#### 'Orders'

Aqui se analisará a tabela de 'orders' (pedidos) para que se entenda o conteto dos dados, a viabilidade deles para o objetivo do projeto e presença de valores ausentes / problemas.

In [18]:
# Visualizando 'orders'

orders.shape
orders.head()
orders.info()
orders.describe(include="all")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   order_id                       99441 non-null  object
 1   customer_id                    99441 non-null  object
 2   order_status                   99441 non-null  object
 3   order_purchase_timestamp       99441 non-null  object
 4   order_approved_at              99281 non-null  object
 5   order_delivered_carrier_date   97658 non-null  object
 6   order_delivered_customer_date  96476 non-null  object
 7   order_estimated_delivery_date  99441 non-null  object
dtypes: object(8)
memory usage: 6.1+ MB


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
count,99441,99441,99441,99441,99281,97658,96476,99441
unique,99441,99441,8,98875,90733,81018,95664,459
top,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2018-08-02 12:05:26,2018-02-27 04:31:10,2018-05-09 15:48:00,2018-05-08 19:36:48,2017-12-20 00:00:00
freq,1,1,96478,3,9,47,3,522


A seção de 'Orders' tem 8 colunas, que descrevem informações de operação quanto aos pedidos, indicando o id do cliente, do vendedor, o status da ordem, a estimativa de entrega, entre outras informações. São 99411 linhas no total, sem valores ausentes, o que dá bastante material para trabalhar com.

O fato da quantidade de order_id únicos serem os mesmos da contagem significa que cada order_id é único, o que é visível na documentação do dataset (o freq = 1 também indica isso.)

Os customers também tem ids únicos, existem 8 tipos de order_status que se repetem, a maioria já sendo 'delivered' (entregue), e outras informações podem ser tiradas por meio da análise da tabela.

In [26]:
orders.isna().mean()

order_id                         0.000000
customer_id                      0.000000
order_status                     0.000000
order_purchase_timestamp         0.000000
order_approved_at                0.001609
order_delivered_carrier_date     0.017930
order_delivered_customer_date    0.029817
order_estimated_delivery_date    0.000000
dtype: float64

Essa visualização destaca os valores nulos, e mede sua proporção. Com isso, dá pra concluir que 0.16% não teve ordem aprovada, 1.7% não tem a data que foi entregue ao transportador e 3% não tem a entregue ao cliente. 

Esses fatores podem ser vindos de cancelamentos, pedidos não entregues, problemas logísticos e falhas nos pagamentos. Essas podem indicar experiências negativas por parte do cliente.

In [29]:
orders['order_id'].nunique() == len(orders)

True

O retorno True do comando revela que todos os order_id são únicos.

In [33]:
orders.merge(reviews, on='order_id', how='left').shape

(99992, 14)

Teste de merge -> Revela explosão de cardinalidade. Número de linhas foi de 99441 para 99992.Isso acontece pois pedidos com mais de um review tiveram linhas duplicadas.

Isso exige cuidado nas análises pois pode causar distorções em certos resultados esperados.

#### 'Reviews'

Aqui observamos a parte do dataset que se refere aos reviews (avaliações) das entregas.

In [22]:
# Visualizando 'reviews'

reviews.shape
reviews.head()
reviews.info()
reviews.describe(include="all")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99224 entries, 0 to 99223
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   review_id                99224 non-null  object
 1   order_id                 99224 non-null  object
 2   review_score             99224 non-null  int64 
 3   review_comment_title     11568 non-null  object
 4   review_comment_message   40977 non-null  object
 5   review_creation_date     99224 non-null  object
 6   review_answer_timestamp  99224 non-null  object
dtypes: int64(1), object(6)
memory usage: 5.3+ MB


,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
count,99224,99224,99224.000000,11568,40977,99224,99224
unique,98410,98673,NaN,4527,36159,636,98248
top,08528f70f579f0c830189efc523d2182,df56136b8031ecd28e200bb18e6ddb2e,NaN,Recomendo,Muito bom,2017-12-19 00:00:00,2017-06-15 23:21:05
freq,3,3,NaN,423,230,463,4
mean,NaN,NaN,4.086421,NaN,NaN,NaN,NaN
std,NaN,NaN,1.347579,NaN,NaN,NaN,NaN
min,NaN,NaN,1.000000,NaN,NaN,NaN,NaN
25%,NaN,NaN,4.000000,NaN,NaN,NaN,NaN
50%,NaN,NaN,5.000000,NaN,NaN,NaN,NaN
75%,NaN,NaN,5.000000,NaN,NaN,NaN,NaN


A tabela de reviews vem com dados das avaliações dadas às experiências. Atualmente, grande parte dos elementos estão como 'NaN', o que revela que essa parte do dataset deve ser investigado para que se entenda o porquê os valores estão ausentes.

Todas as variáveis exceto review_score são tratadas como objects, enquanto review_score como int64. Isso revela o porquê do NaN aparecer quando tenta se fazer cálculos com objects e aparece quando tenta se medir a unique, top e freq de review_score (int64).

Isso acontece pela especificação de 'include all' em review.describe(), que não nota que mean, std, max e min só fazem sentido para o review_score.

In [27]:
reviews.isna().mean()

review_id                  0.000000
order_id                   0.000000
review_score               0.000000
review_comment_title       0.883415
review_comment_message     0.587025
review_creation_date       0.000000
review_answer_timestamp    0.000000
dtype: float64

Essa visualização revela que 88% das reviews estão sem título e 58% não tem corpo de texto.

In [34]:
reviews['review_id'].nunique() == len(reviews)

False

A característica de 'False' após verificação de se cada linha tem um id única evidencia que podem haver mais de uma linha compartilhando o mesmo id. Talvez por uma atualização do mesmo review, ou outras questões que trazem consigo um alerta para a forma em que os dados serão analisados para evitar interpretações errônias.

#### 'Customers'

Analisa a parte dos dados que trata dos clientes.

In [23]:
# Visualizando 'customers'

customers.shape
customers.head()
customers.info()
customers.describe(include="all")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               99441 non-null  object
 1   customer_unique_id        99441 non-null  object
 2   customer_zip_code_prefix  99441 non-null  int64 
 3   customer_city             99441 non-null  object
 4   customer_state            99441 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.8+ MB


,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
count,99441,99441,99441.000000,99441,99441
unique,99441,96096,NaN,4119,27
top,06b8999e2fba1a1fbc88172c00ba8bc7,8d50f5eadf50201ccdcedfb9e2ac8455,NaN,sao paulo,SP
freq,1,17,NaN,15540,41746
mean,NaN,NaN,35137.474583,NaN,NaN
std,NaN,NaN,29797.938996,NaN,NaN
min,NaN,NaN,1003.000000,NaN,NaN
25%,NaN,NaN,11347.000000,NaN,NaN
50%,NaN,NaN,24416.000000,NaN,NaN
75%,NaN,NaN,58900.000000,NaN,NaN


A parte de 'Customers' repete o padrão de reviews de mostrar dados 'NaN' pra o que não faz sentido de acordo com os tipos de variáveis. Essa parte dá informações sobre o cliente, dando mais prioridade à seu endereço e código postal.

In [28]:
customers.isna().mean()

customer_id                 0.0
customer_unique_id          0.0
customer_zip_code_prefix    0.0
customer_city               0.0
customer_state              0.0
dtype: float64

A visualização indica que nenhum cliente está com qualquer das variáveis nulas (o que não significa necessariamente que todos os endereços estão corretos ou que a expectativa do cliente está sendo necessariamente atendida)

In [35]:
customers['customer_id'].nunique() == len(customers)

True

Da mesma forma, todo cliente tem seu próprio Id.

### Achados e considerações

Com essas observações, consegue-se entender muito mais sobre o dataset e levantar algumas hipóteses para o desenvolvimento das análises.

- A tabela de 'orders' tem ids únicos que não se repetem.
- A tabela de 'reviews' repete registros por pedido.
- Isso causa que merges de pedidos e avaliações duplique linhas.
- Valores ausentes encontrados em dados deentrega e operacionais, revelando cancelamentos ou pedidos não entregues.

### Implicações

Merges simples não poderão ser feitos pois podem eviesar os resultados. Serão necessárias a criação de regras de agregação e tratamento. Esses serão os próximos passos do projeto.